In [1]:
%load_ext cython

In [2]:
import os
import numpy as np
import cPickle as pickle
import random as rn
import time
import math as m


In [3]:

class vocab_word:
    #trace from the root
    point=[]
    #huffman code
    code=[]
    def __init__(self, word, count):
        self.count = count
        self.word = word
     
    #object print 
    def __repr__(self):
        return repr(( self.word,self.count))

In [11]:
threshold = 50000

In [12]:

vocab = pickle.load( open( "vocab.p", "rb" ) )

syn0=pickle.load( open( "WeightMatrix.p", "rb" ) )
#CreateBinaryTree()

#make it all uppercase 
for i in xrange(len(vocab)):
    vocab[i].word=vocab[i].word.upper()




vocab[:]=vocab[:threshold]
vocab_size=len(vocab)

idx={}
invidx={}
for i in xrange(vocab_size):
    if(vocab[i].word in idx):continue 
    idx[vocab[i].word]=i
    
#save word index and weight matrix
for i in xrange(vocab_size):
    invidx[i]=vocab[i].word
    


In [13]:
#print  top k  
def print_topk(query,matrix,idx,inv_idx,k):
    rank={}
    topk=[]
    query=query.upper()
    for i in xrange(threshold):
        if(i==idx[query]):continue 
        sim=np.dot(matrix[idx[query]],matrix[i])
        rank[i]=sim
    topk=sorted(rank.items(), key=lambda x: x[1],reverse=True)
    
    for i in xrange(k):
        print (inv_idx[topk[i][0]],topk[i][1])
    


In [14]:

def normalize(weight):
    for i in xrange(threshold):
        norm=0
        for j in xrange(weight.shape[1]):
            norm+=weight[i][j]**2
        norm=m.sqrt(norm)
        weight[i]=weight[i]/float(norm)
        
        
            

In [16]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
threshold=50000

def normalize_accel(np.ndarray[np.float64_t, ndim=2] weight):
    cdef float norm=0
    for i in xrange(threshold):
        norm=0
        for j in xrange(weight.shape[1]):
            norm+=weight[i][j]**2
        norm=norm**(0.5)
        weight[i]=weight[i]/float(norm)
        

In [17]:

def normalize_vec(vector): 
    norm=0
    for j in xrange(len(vector)):
        norm+=vector[j]**2
    
    norm=norm**0.5
    vector=vector/float(norm)
    return vector 



In [18]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

def normalize_vec_accel(np.ndarray[np.float64_t, ndim=1] vector): 
    cdef float norm=0
    for j in xrange(len(vector)):
        norm+=vector[j]**2
    
    norm=norm**0.5
    vector=vector/float(norm)
    return vector 



In [20]:
%%time
normalize(syn0)

CPU times: user 1.03 s, sys: 0 ns, total: 1.03 s
Wall time: 1.03 s


In [21]:
%%time
normalize_accel(syn0)



CPU times: user 944 ms, sys: 52 ms, total: 996 ms
Wall time: 920 ms


In [22]:
norm=0
for j in xrange(syn0.shape[1]):
    norm+=syn0[1][j]**2
print(norm)

1.0


In [23]:
type(syn0)

numpy.ndarray

In [24]:
def analogytask(vector,idx,inv_idx):
    maxim=.0
    maxword=''
    accuracy=0
    count=0
    
    for i in xrange(threshold):
        sim=np.dot(vector,syn0[i])
        if(maxim<sim):
            maxim=sim
            maxword=inv_idx[i]
            
    return maxword

In [25]:
def analogytask_topk(vector,idx,inv_idx,k):
    rank={}
    topk=[]
    for i in xrange(threshold):
        sim=np.dot(vector,syn0[i])
        rank[i]=sim
    topk=sorted(rank.items(), key=lambda x: x[1],reverse=True)
    
    return [inv_idx[topk[i][0]] for i in xrange(k)]
 

In [35]:
%%time
vec=normalize_vec(syn0[idx['man'.upper()]]-syn0[idx['king'.upper()]]+syn0[idx['queen'.upper()]])
vec2=syn0[idx['capital'.upper()]]+syn0[idx['japan'.upper()]]
analogytask_topk(vec,idx,invidx,4)


CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 151 ms


In [37]:
analogytask_topk(vec,idx,invidx,4)

['MAN', 'WOMAN', 'DOG', 'GIRL']

In [27]:
%%time
vec=normalize_vec(syn0[idx['korean'.upper()]]-syn0[idx['korea'.upper()]]+syn0[idx['japan'.upper()]])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 57.2 µs


In [28]:
%%time
vec=normalize_vec_accel(syn0[idx['korean'.upper()]]-syn0[idx['korea'.upper()]]+syn0[idx['japan'.upper()]])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 35 µs


In [29]:
def evaluation(matrix,idx,inv_idx):
    with open('../data/questions-words.txt', 'rb') as train_file:
        train_file.readline()
        totalcount = 0
        words=[]
        result=[]
        accur=0
        for line in train_file:
            words=line.split()
            if(len(words)<3):
                continue
                
            words[0]=words[0].upper()
            words[1]=words[1].upper()
            words[2]=words[2].upper()
            words[3]=words[3].upper()
            
            not_include=words[:3]
            
            #same as the given word continue 
            if(words[0] not in idx or words[1] not in idx or words[2] not in idx or words[3] not in idx):
                continue
            
            
            lin_cal=matrix[idx[words[1]]]-matrix[idx[words[0]]]+matrix[idx[words[2]]]
            lin_cal=normalize_vec_accel(lin_cal)
            result=analogytask_topk(lin_cal,idx,invidx,4)
            
            for i in xrange(len(result)):           
                if(result[i] in not_include):
                    continue 
                if(result[i]==words[3]):
                    accur+=1
                    break  
                if(result[i] not in words):
                    break        
            totalcount+=1
        print(totalcount)
        print(accur)
        print(accur/float(totalcount))



In [32]:
%%time
evaluation(syn0,idx,invidx)

15790
1001
0.0633945535149
CPU times: user 23min 18s, sys: 560 ms, total: 23min 19s
Wall time: 23min 18s


In [31]:
print_topk('jazz',syn0,idx,invidx,10)

('POP', 0.93122598661943123)
('BLUES', 0.92697214812822215)
('PUNK', 0.92615348364900951)
('RAP', 0.92420971046826317)
('HOP', 0.91652583386905406)
('DISCO', 0.90802467965995015)
('FOLK', 0.90742165525090535)
('MUSIC', 0.89954854602555612)
('MUSICIANS', 0.89745459325992571)
('HIP', 0.89445943635573333)
